In [14]:
import warnings
import graphviz
import re
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.tools.eval_measures import rmse
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성 패키지 불러오기
from sklearn.linear_model import LinearRegression # sklearn 선형회귀 모형 -> 변수 선택법을 위함
from sklearn.feature_selection import RFE # 변수 선택법 (후진 제거법)
from sklearn.preprocessing import scale # 데이터 scale
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score
# Import ML Classification algos
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
warnings.filterwarnings('ignore')
%matplotlib inline
matplotlib.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus']=False

In [15]:
ds_asso= [["소주","콜라","맥주"],
         ["소주","콜라","와인"],
         ["소주","주스"],
         ["콜라","맥주"],
         ["소주","콜라","맥주","와인"],
         ["주스"]]
ds_asso

[['소주', '콜라', '맥주'],
 ['소주', '콜라', '와인'],
 ['소주', '주스'],
 ['콜라', '맥주'],
 ['소주', '콜라', '맥주', '와인'],
 ['주스']]

In [16]:
# items 중 index 0번이 갖고 있으면 True 아니면 False
enc = TransactionEncoder()
arr =enc.fit_transform(X=ds_asso)
ds_pre=pd.DataFrame(arr,columns=enc.columns_)
ds_pre

,맥주,소주,와인,주스,콜라
0,True,True,False,False,True
1,False,True,True,False,True
2,False,True,False,True,False
3,True,False,False,False,True
4,True,True,True,False,True
5,False,False,False,True,False


In [17]:
# 최소 지지도
min_support = 0.5
# 지지도 계산
ds_freq = apriori(ds_pre,min_support =0.5,use_colnames=True)
ds_freq.round(3)

,support,itemsets
0,0.500,(맥주)
1,0.667,(소주)
2,0.667,(콜라)
3,0.500,"(콜라, 맥주)"
4,0.500,"(콜라, 소주)"


In [18]:
ds_asso_rule=association_rules(ds_freq,metric="confidence",min_threshold=0.5)
ds_asso_rule.round(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(콜라),(맥주),0.667,0.500,0.5,0.75,1.500,0.167,2.000000
1,(맥주),(콜라),0.500,0.667,0.5,1.00,1.500,0.167,inf
2,(콜라),(소주),0.667,0.667,0.5,0.75,1.125,0.056,1.333000
3,(소주),(콜라),0.667,0.667,0.5,0.75,1.125,0.056,1.333000
